### **Storing on the NFS**

This notebook will show how to connect to an exisiting file system and write/read to that FS. To connect, a persistent volume claim has to be set up to you namespace. The files written to the NFS are permanent and passable between workflows.

**First some imports and global settings**

In [ ]:
from hera.workflows import script,  DAG, Workflow
from hera.shared import global_config

global_config.host = "https://dev.services.eodc.eu/workflows/"
global_config.namespace = "<YOUR NAMESPACE>"
global_config.token = "<YOUR TOKEN>"
global_config.image = "ghcr.io/eodcgmbh/cluster_image:2025.2.0"

**Writing scripts**

In [ ]:
@script()
def write_message(message):
    print(message)

**Creating the Workflow**

We can pass multiple values for the parameter in our functions by using `with_items`, the workflow will then work in parallel for every value passed to the function. This also works if multiple parameters should be given by adding keys and values to the dict passed to `with_items`.

In [ ]:
with Workflow(
    generate_name="loop-",
    entrypoint="workflow"
) as w:
    with DAG(name="workflow"):
        write_message(arguments={"message": "{{item.msg}}"},
                      with_items=[{"msg": "Hello1"}, {"msg": "Hello2"}, {"msg": "Hello3"}])

The same can be achieved by passing a list to the `with_params` keyword.

In [ ]:
parameters = ["Hello1", "Hello2", "Hello3"]

with Workflow(
    generate_name="loop2-",
    entrypoint="workflow"
) as w2:
    with DAG(name="workflow"):
        write_message(arguments={"message": "{{item}}"},
                      with_param=parameters)

**Submitting the Workflow**

In [ ]:
w.create()
w2.create()